In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_csv('data/partd_heme-onc_16.txt', delimiter='\t')
cols = pd.read_csv('data/CMS_PartD_Prescriber_NPI_Drug_CY2016_100.txt', delimiter='\t').columns
df.columns = cols

df.groupby('drug_name').total_claim_count.sum().sort_values(ascending=False)[:20]

drug_name
ANASTROZOLE                  624578
LETROZOLE                    288358
TAMOXIFEN CITRATE            204889
HYDROCODONE-ACETAMINOPHEN    198792
DEXAMETHASONE                193132
HYDROXYUREA                  177111
PREDNISONE                   177009
ONDANSETRON HCL              167238
WARFARIN SODIUM              143918
PROCHLORPERAZINE MALEATE     140402
REVLIMID                     138947
OXYCODONE HCL                137541
GABAPENTIN                   127694
ACYCLOVIR                    108530
EXEMESTANE                   102476
LORAZEPAM                     98193
OMEPRAZOLE                    91267
XARELTO                       90100
POTASSIUM CHLORIDE            82221
OXYCODONE-ACETAMINOPHEN       79274
Name: total_claim_count, dtype: int64

In [ ]:
GLEEVEC, REVLIMID, IBRANCE, JAKAFI, TARCEVA, TASIGNA, SPRYCEL, IMATINIB MESYLATE 

In [59]:
df2 = pd.read_csv('data/partb_heme-onc_16.txt', delimiter='\t')
cols2 = pd.read_csv('data/CMS_PartB_Provider_Util_Payment_CY2016_100.txt', delimiter='\t').columns
df2.columns = cols2

LSC = pd.DataFrame(df2.groupby('HCPCS_CODE')['LINE_SRVC_CNT'].sum().sort_values(ascending=False))
LSC.reset_index(inplace=True)
df3 = pd.merge(LSC, df2[['HCPCS_CODE', 'HCPCS_DESCRIPTION']], on='HCPCS_CODE', how='left')
df3 = df3.drop_duplicates().reset_index().drop('index', axis=1)
df3.head()

,HCPCS_CODE,LINE_SRVC_CNT,HCPCS_DESCRIPTION
0,J1439,29870319.0,"Injection, ferric carboxymaltose, 1 mg"
1,J0881,24118655.0,"Injection, darbepoetin alfa, 1 microgram (non-..."
2,J1442,22222299.0,"Injection, filgrastim (g-csf), excludes biosim..."
3,Q0138,19130003.0,"Injection, ferumoxytol, for treatment of iron ..."
4,J0897,12392081.0,"Injection, denosumab, 1 mg"


In [21]:
trastuzumab, paclitaxel, bevacizumab, bortezomib, nivolumab

Index(['NPI', 'NPPES_PROVIDER_LAST_ORG_NAME', 'NPPES_PROVIDER_FIRST_NAME',
       'NPPES_PROVIDER_MI', 'NPPES_CREDENTIALS', 'NPPES_PROVIDER_GENDER',
       'NPPES_ENTITY_CODE', 'NPPES_PROVIDER_STREET1', 'NPPES_PROVIDER_STREET2',
       'NPPES_PROVIDER_CITY', 'NPPES_PROVIDER_ZIP', 'NPPES_PROVIDER_STATE',
       'NPPES_PROVIDER_COUNTRY', 'PROVIDER_TYPE',
       'MEDICARE_PARTICIPATION_INDICATOR', 'PLACE_OF_SERVICE', 'HCPCS_CODE',
       'HCPCS_DESCRIPTION', 'HCPCS_DRUG_INDICATOR', 'LINE_SRVC_CNT',
       'BENE_UNIQUE_CNT', 'BENE_DAY_SRVC_CNT', 'AVERAGE_MEDICARE_ALLOWED_AMT',
       'AVERAGE_SUBMITTED_CHRG_AMT', 'AVERAGE_MEDICARE_PAYMENT_AMT',
       'AVERAGE_MEDICARE_STANDARD_AMT'],
      dtype='object')

In [ ]:
GLEEVEC, REVLIMID, IBRANCE, JAKAFI, TARCEVA, TASIGNA, SPRYCEL, IMATINIB MESYLATE 

In [80]:
drugs = ['GLEEVEC', 'REVLIMID', 'IBRANCE', 'JAKAFI', 'TARCEVA', 'TASIGNA', 'SPRYCEL', 'IMATINIB MESYLATE']
#drugs = ['GLEEVEC']
partd_df = pd.DataFrame(columns=['npi'])
for drug in drugs:
    drug_df = df.loc[df['drug_name'] == drug, ['npi', 'total_claim_count']]
    drug_df.columns = ['npi', drug]
    partd_df = pd.merge(partd_df, drug_df, on='npi', how='outer')
    
partd_df.fillna(0, inplace=True)
partd_df.head()

,npi,GLEEVEC,REVLIMID,IBRANCE,JAKAFI,TARCEVA,TASIGNA,SPRYCEL,IMATINIB MESYLATE
0,1003803685,21.0,18.0,0.0,12.0,0.0,0.0,29.0,0.0
1,1003804204,15.0,59.0,18.0,0.0,11.0,0.0,0.0,0.0
2,1003804287,14.0,107.0,0.0,0.0,0.0,0.0,0.0,33.0
3,1003807355,16.0,41.0,0.0,0.0,0.0,19.0,0.0,0.0
4,1003810383,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
from sklearn.cluster import KMeans

In [85]:
km = KMeans(n_clusters=8)
km.fit(partd_df)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=8, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [ ]:
# Create PCA plot of clusters